In [ ]:
# ====================================================================
# WEEK 5: STATISTICAL ANALYSIS
# DNA Gene Mapping Project
# Author: Sharique Mohammad
# Date: 11 January 2026
# ====================================================================
# jupyter_notebooks/01_statistical_analysis.ipynb
# ====================================================================

In [ ]:
# CELL 0: Create Required Folders
import os

# Create folders if they don't exist
os.makedirs('data/analytical/visualizations', exist_ok=True)
os.makedirs('data/analytical', exist_ok=True)
os.makedirs('data/ml', exist_ok=True)

print("Folders created successfully!")
print("- data/analytical/")
print("- data/analytical/visualizations/")
print("- data/ml/")

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import os
import warnings

In [ ]:
warnings.filterwarnings('ignore')

load_dotenv()

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully")

In [ ]:
# Database Connection

DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'port': int(os.getenv('POSTGRES_PORT', 5432)),
    'database': os.getenv('POSTGRES_DATABASE', 'genome_db'),
    'user': os.getenv('POSTGRES_USER', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD')
}

engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}"
    f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

conn = psycopg2.connect(
    host=DB_CONFIG['host'],
    database=DB_CONFIG['database'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password']
)

print("Database connection established")

In [ ]:
# Load Data from PostgreSQL

print("Loading data from PostgreSQL gold layer...")

df_gene_features = pd.read_sql("SELECT * FROM gold.gene_features", engine)
df_chromosome_features = pd.read_sql("SELECT * FROM gold.chromosome_features", engine)
df_gene_disease = pd.read_sql("SELECT * FROM gold.gene_disease_association", engine)
df_ml_features = pd.read_sql("SELECT * FROM gold.ml_features", engine)

print(f"Loaded {len(df_gene_features)} genes")
print(f"Loaded {len(df_chromosome_features)} chromosomes")
print(f"Loaded {len(df_gene_disease)} gene-disease associations")
print(f"Loaded {len(df_ml_features)} ML features")

In [ ]:
# Data Overview

print("\n" + "="*70)
print("GENE FEATURES OVERVIEW")
print("="*70)
print(df_gene_features.head())
print("\nData Types:")
print(df_gene_features.dtypes)
print("\nBasic Statistics:")
print(df_gene_features.describe())

In [ ]:
# Missing Value Analysis

print("\n" + "="*70)
print("MISSING VALUE ANALYSIS")
print("="*70)

missing_data = pd.DataFrame({
    'Column': df_gene_features.columns,
    'Missing_Count': df_gene_features.isnull().sum(),
    'Missing_Percentage': (df_gene_features.isnull().sum() / len(df_gene_features) * 100).round(2)
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_data) > 0:
    print("\nColumns with missing values:")
    print(missing_data)
else:
    print("\nNo missing values found")

In [ ]:
# Distribution Analysis - Mutation Counts

print("\n" + "="*70)
print("DISTRIBUTION ANALYSIS - MUTATION COUNTS")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Histogram
axes[0, 0].hist(df_gene_features['mutation_count'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of Mutation Counts', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Mutation Count')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df_gene_features['mutation_count'].median(), color='red', 
                   linestyle='--', label='Median')
axes[0, 0].legend()

# Box plot
axes[0, 1].boxplot(df_gene_features['mutation_count'], vert=True)
axes[0, 1].set_title('Mutation Count Box Plot', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Mutation Count')

# Q-Q plot
stats.probplot(df_gene_features['mutation_count'], dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot: Mutation Counts', fontsize=14, fontweight='bold')

# Log-transformed histogram
log_mutations = np.log1p(df_gene_features['mutation_count'])
axes[1, 1].hist(log_mutations, bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].set_title('Log-Transformed Mutation Counts', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Log(Mutation Count + 1)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('data/analytical/visualizations/mutation_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nMutation Count Statistics:")
print(f"  Mean: {df_gene_features['mutation_count'].mean():.2f}")
print(f"  Median: {df_gene_features['mutation_count'].median():.2f}")
print(f"  Std Dev: {df_gene_features['mutation_count'].std():.2f}")
print(f"  Min: {df_gene_features['mutation_count'].min()}")
print(f"  Max: {df_gene_features['mutation_count'].max()}")

# Normality test
stat, p_value = stats.shapiro(df_gene_features['mutation_count'][:50])
print(f"\nShapiro-Wilk Test for Normality:")
print(f"  Statistic: {stat:.4f}")
print(f"  P-value: {p_value:.4f}")
if p_value < 0.05:
    print("  Result: Data is NOT normally distributed")
else:
    print("  Result: Data appears normally distributed")


In [ ]:
# Distribution Analysis - Pathogenic Ratio

print("\n" + "="*70)
print("DISTRIBUTION ANALYSIS - PATHOGENIC RATIO")
print("="*70)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Histogram
axes[0].hist(df_gene_features['pathogenic_ratio'], bins=30, edgecolor='black', alpha=0.7, color='coral')
axes[0].set_title('Distribution of Pathogenic Ratios', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Pathogenic Ratio')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df_gene_features['pathogenic_ratio'].median(), color='red', 
               linestyle='--', label='Median')
axes[0].legend()

# Box plot by risk level
risk_order = ['Low', 'Medium', 'High']
df_gene_features['risk_level'] = pd.Categorical(df_gene_features['risk_level'], 
                                                  categories=risk_order, 
                                                  ordered=True)
sns.boxplot(data=df_gene_features, x='risk_level', y='pathogenic_ratio', ax=axes[1])
axes[1].set_title('Pathogenic Ratio by Risk Level', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Risk Level')
axes[1].set_ylabel('Pathogenic Ratio')

plt.tight_layout()
plt.savefig('data/analytical/visualizations/pathogenic_ratio_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nPathogenic Ratio Statistics:")
print(f"  Mean: {df_gene_features['pathogenic_ratio'].mean():.4f}")
print(f"  Median: {df_gene_features['pathogenic_ratio'].median():.4f}")
print(f"  Std Dev: {df_gene_features['pathogenic_ratio'].std():.4f}")


In [ ]:
# Hypothesis Test 1 - ANOVA (Chromosomes vs Mutation Counts)

print("\n" + "="*70)
print("HYPOTHESIS TEST 1: ANOVA")
print("Do different chromosomes have significantly different mutation counts?")
print("="*70)

# Prepare data by chromosome
chromosome_groups = []
for chrom in df_gene_features['chromosome'].dropna().unique():
    group_data = df_gene_features[df_gene_features['chromosome'] == chrom]['mutation_count'].values
    if len(group_data) > 0:
        chromosome_groups.append(group_data)

# Perform ANOVA
f_statistic, p_value = stats.f_oneway(*chromosome_groups)

print(f"\nANOVA Results:")
print(f"  F-statistic: {f_statistic:.4f}")
print(f"  P-value: {p_value:.4f}")
print(f"  Alpha level: 0.05")

if p_value < 0.05:
    print(f"\nConclusion: REJECT null hypothesis")
    print(f"  Chromosomes have significantly different mutation counts (p < 0.05)")
else:
    print(f"\nConclusion: FAIL TO REJECT null hypothesis")
    print(f"  No significant difference in mutation counts across chromosomes")

# Visualization
plt.figure(figsize=(14, 6))
sns.boxplot(data=df_gene_features, x='chromosome', y='mutation_count', order=sorted(df_gene_features['chromosome'].unique()))
plt.title('Mutation Count Distribution by Chromosome', fontsize=14, fontweight='bold')
plt.xlabel('Chromosome')
plt.ylabel('Mutation Count')
plt.xticks(rotation=0)
plt.savefig('data/analytical/visualizations/chromosome_mutation_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Hypothesis Test 2 - Chi-Square (Risk Level vs Chromosome)

print("\n" + "="*70)
print("HYPOTHESIS TEST 2: CHI-SQUARE TEST")
print("Is there an association between chromosome and risk level?")
print("="*70)

# Create contingency table
contingency_table = pd.crosstab(df_gene_features['chromosome'], df_gene_features['risk_level'])
print("\nContingency Table:")
print(contingency_table)

# Perform chi-square test
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"\nChi-Square Test Results:")
print(f"  Chi-square statistic: {chi2:.4f}")
print(f"  P-value: {p_value:.4f}")
print(f"  Degrees of freedom: {dof}")
print(f"  Alpha level: 0.05")

if p_value < 0.05:
    print(f"\nConclusion: REJECT null hypothesis")
    print(f"  There IS a significant association between chromosome and risk level")
else:
    print(f"\nConclusion: FAIL TO REJECT null hypothesis")
    print(f"  No significant association between chromosome and risk level")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

contingency_table.plot(kind='bar', stacked=True, ax=axes[0])
axes[0].set_title('Risk Level Distribution by Chromosome (Stacked)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Chromosome')
axes[0].set_ylabel('Count')
axes[0].legend(title='Risk Level')
axes[0].tick_params(axis='x', rotation=0)

sns.heatmap(contingency_table, annot=True, fmt='d', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('Contingency Table Heatmap', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Risk Level')
axes[1].set_ylabel('Chromosome')

plt.tight_layout()
plt.savefig('data/analytical/visualizations/risk_chromosome_association.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Hypothesis Test 3 - T-Test (High vs Low Risk)

print("\n" + "="*70)
print("HYPOTHESIS TEST 3: INDEPENDENT T-TEST")
print("Do High-risk genes have significantly more mutations than Low-risk genes?")
print("="*70)

high_risk = df_gene_features[df_gene_features['risk_level'] == 'High']['mutation_count']
low_risk = df_gene_features[df_gene_features['risk_level'] == 'Low']['mutation_count']

print(f"High-risk genes: n={len(high_risk)}, mean={high_risk.mean():.2f}")
print(f"Low-risk genes: n={len(low_risk)}, mean={low_risk.mean():.2f}")

# Perform t-test
t_statistic, p_value = stats.ttest_ind(high_risk, low_risk)

print(f"\nT-Test Results:")
print(f"  T-statistic: {t_statistic:.4f}")
print(f"  P-value: {p_value:.4f}")
print(f"  Alpha level: 0.05")

if p_value < 0.05:
    print(f"\nConclusion: REJECT null hypothesis")
    print(f"  High-risk genes have significantly different mutation counts than Low-risk genes")
else:
    print(f"\nConclusion: FAIL TO REJECT null hypothesis")
    print(f"  No significant difference in mutation counts")

# Visualization
plt.figure(figsize=(10, 6))
data_to_plot = [high_risk, low_risk]
plt.boxplot(data_to_plot, labels=['High Risk', 'Low Risk'], patch_artist=True)
plt.title('Mutation Count: High Risk vs Low Risk Genes', fontsize=14, fontweight='bold')
plt.ylabel('Mutation Count')
plt.grid(axis='y', alpha=0.3)
plt.savefig('data/analytical/visualizations/high_vs_low_risk_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation Analysis

print("\n" + "="*70)
print("CORRELATION ANALYSIS")
print("="*70)

# Select numeric columns
numeric_cols = ['mutation_count', 'pathogenic_count', 'total_pathogenic', 
                'pathogenic_ratio', 'disease_count', 'risk_score']
correlation_data = df_gene_features[numeric_cols].dropna()

# Calculate correlation matrix
correlation_matrix = correlation_data.corr()

print("\nCorrelation Matrix:")
print(correlation_matrix.round(3))

# Visualization
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Gene Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('data/analytical/visualizations/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Find strong correlations
print("\nStrong Correlations (|r| > 0.7):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            print(f"  {correlation_matrix.columns[i]} <-> {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")


In [ ]:
# Disease Association Analysis

print("\n" + "="*70)
print("DISEASE ASSOCIATION ANALYSIS")
print("="*70)

# Top diseases by gene count
top_diseases = df_gene_disease.groupby('disease').size().sort_values(ascending=False).head(10)
print("\nTop 10 Diseases by Gene Count:")
print(top_diseases)

# Visualization
plt.figure(figsize=(12, 8))
top_diseases.plot(kind='barh', color='steelblue')
plt.title('Top 10 Diseases by Number of Associated Genes', fontsize=14, fontweight='bold')
plt.xlabel('Number of Genes')
plt.ylabel('Disease')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('data/analytical/visualizations/top_diseases.png', dpi=300, bbox_inches='tight')
plt.show()

# Association strength distribution
print("\nAssociation Strength Distribution:")
print(df_gene_disease['association_strength'].value_counts())

plt.figure(figsize=(10, 6))
df_gene_disease['association_strength'].value_counts().plot(kind='bar', color=['green', 'orange', 'red'])
plt.title('Gene-Disease Association Strength Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Association Strength')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('data/analytical/visualizations/association_strength.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Summary Statistics Report

print("\n" + "="*70)
print("STATISTICAL ANALYSIS SUMMARY REPORT")
print("="*70)

summary_report = {
    'Total Genes Analyzed': len(df_gene_features),
    'Total Chromosomes': df_gene_features['chromosome'].nunique(),
    'Total Diseases': len(df_gene_disease['disease'].unique()),
    'Mean Mutations per Gene': df_gene_features['mutation_count'].mean(),
    'Median Mutations per Gene': df_gene_features['mutation_count'].median(),
    'Mean Pathogenic Ratio': df_gene_features['pathogenic_ratio'].mean(),
    'High Risk Genes': len(df_gene_features[df_gene_features['risk_level'] == 'High']),
    'Medium Risk Genes': len(df_gene_features[df_gene_features['risk_level'] == 'Medium']),
    'Low Risk Genes': len(df_gene_features[df_gene_features['risk_level'] == 'Low'])
}

for key, value in summary_report.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Save Analysis Results

print("\n" + "="*70)
print("SAVING ANALYSIS RESULTS")
print("="*70)

# Save summary statistics
summary_df = pd.DataFrame([summary_report]).T
summary_df.columns = ['Value']
summary_df.to_csv('data/analytical/summary_statistics.csv')
print("Saved: data/analytical/summary_statistics.csv")

# Save correlation matrix
correlation_matrix.to_csv('data/analytical/correlation_matrix.csv')
print("Saved: data/analytical/correlation_matrix.csv")

print("\nStatistical Analysis Complete!")
print("Visualizations saved in: data/analytical/visualizations/")

# Close database connection
conn.close()
print("\nDatabase connection closed")